In [1]:
""" 
Run nSimplices + MDS vs MDS on subset (STOOL and VAGINA only) of HMP dataset 
"""

' \nRun nSimplices + MDS vs MDS on subset (STOOL and VAGINA only) of HMP dataset \n'

In [2]:
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import os
import pandas as pd
import random as alea
from scipy.spatial.distance import pdist, squareform
from sklearn.manifold import MDS


data_dir = "../data/"
output_dir = "../outputs"
target_sites = ['STOOL', 'VAGINA']
target_colors = ["cornflowerblue", "orange"]

In [3]:
"""
Prepare dataset
"""

# iterate over QE and NB
files = ["hmp_v13lqphylotypeQuantNB_rs_c.csv", "hmp_v13lqphylotypeQuantE_rs_c.csv"]
color_df = pd.read_csv(os.path.join(data_dir, "hmp_v13lqphylotypePheno_rs_c.csv"), header=0)

for file in files:
    data_path = os.path.join(data_dir, file)
    df_hmp_ori = np.loadtxt(data_path, delimiter=",")
    df_hmp = []


    for index, row in color_df.iterrows():
        site_exist = False
        for site in target_sites:
            if row[site]:
                site_exist = True
        if site_exist:
            df_hmp.append(df_hmp_ori[index])
    df_hmp = np.array(df_hmp)
    np.savetxt(os.path.join(data_dir, file[:-8]+"target_"+file[-8:]), df_hmp, fmt='%1.17f', delimiter=',')


In [4]:
""" 
Prepare colors
"""

colors = []
new_color_df = color_df.copy(deep = True)
drop_indices = []

for index, row in color_df.iterrows():
    site_exist = False
    for i in range(len(target_sites)):
        site = target_sites[i]
        if row[site]:
            colors.append(target_colors[i])
            site_exist = True
    if not site_exist:
        drop_indices.append(index)

colors = np.array(colors)
print(colors.shape)

np.savetxt(os.path.join(data_dir, "hmp_target_colors.txt"), colors, fmt="%s")

new_color_df = new_color_df.drop(drop_indices)
new_color_df.to_csv(os.path.join(data_dir, "hmp_v13lqphylotypePheno_target_rs_c.csv"), header=True, index=False)

(330,)


In [5]:
""" 
Run nSimplices on HMP dataset
"""
colors = np.loadtxt(os.path.join(data_dir, "hmp_target_colors.txt"), dtype="str")
exec(open("../nsimplices.py").read())
alea.seed(42)


In [8]:
""" 
Run
(1) NB normalization + nSimplices + cMDS 
(2) QE normalization + nSimplices + cMDS 

To derive the axes data
""" 

output_files = ["hmp_NB_nSimplices_target_cMDS_axes.txt", "hmp_QE_nSimplices_target_cMDS_axes.txt"]
data_files = ["hmp_v13lqphylotypeQuantNB_target_rs_c.csv", "hmp_v13lqphylotypeQuantE_target_rs_c.csv"]
subspace_dims = []
for i in range(len(output_files)):
    output_file = output_files[i]
    data_file = data_files[i]
    axes_output_path = os.path.join(output_dir, output_file)
    print("======== NB/QE normalization + nSimplices + cMDS ========")
    data_path = os.path.join(data_dir, data_file)
    df_hmp = np.loadtxt(data_path, delimiter=",")
    hmp_dis_sq=squareform(pdist(df_hmp))

    feature_num = 11
    dim_start = 2
    dim_end = 50

    print("hmp_dis_sq shape is:", hmp_dis_sq.shape)
    outlier_indices, subspace_dim , corr_pairwise_dis, corr_coord = nsimplices(hmp_dis_sq, feature_num, dim_start, dim_end)
    print("subspace dimension is:", subspace_dim)
    subspace_dims.append(subspace_dim)

    # run cMDS to get the corrected coordinates in importance decreasing order
    _, _, Xe = cMDS(corr_pairwise_dis)
    np.savetxt(axes_output_path, Xe, fmt='%f')

======== NB/QE normalization + nSimplices + cMDS ========
hmp_dis_sq shape is: (330, 330)
dim in find_subspace_dim is: 2
dim in find_subspace_dim is: 3
dim in find_subspace_dim is: 4
dim in find_subspace_dim is: 5
dim in find_subspace_dim is: 6
dim in find_subspace_dim is: 7
dim in find_subspace_dim is: 8
dim in find_subspace_dim is: 9
dim in find_subspace_dim is: 10
dim in find_subspace_dim is: 11
dim in find_subspace_dim is: 12
dim in find_subspace_dim is: 13
dim in find_subspace_dim is: 14
dim in find_subspace_dim is: 15
dim in find_subspace_dim is: 16
dim in find_subspace_dim is: 17
dim in find_subspace_dim is: 18
dim in find_subspace_dim is: 19
dim in find_subspace_dim is: 20
dim in find_subspace_dim is: 21
dim in find_subspace_dim is: 22
dim in find_subspace_dim is: 23
dim in find_subspace_dim is: 24
dim in find_subspace_dim is: 25
dim in find_subspace_dim is: 26
dim in find_subspace_dim is: 27
dim in find_subspace_dim is: 28
dim in find_subspace_dim is: 29
dim in find_subspace_d

/home/wanxinli/.local/lib/python3.9/site-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


outliet_indices is: [  0  12  44  67  75 148 152 199 211 267 275 279 287 288 304 318]
original coord is: [ 0.15984135  0.0036035  -0.37747843 -0.48248734  0.70804701  0.25165488
 -1.24051535 -0.03175118 -2.07719903 -1.06425853 -0.5580024 ]
proj_coord is: [ 0.07019633  0.20188897 -0.086807   -0.04296213  0.61629732  0.26242933
  0.00643937  0.25492632 -0.21334472 -0.0082669   0.40451734]
proj_coord is: [ 0.0184052   0.18707489 -0.15986685  0.10114205  0.66774857  0.21413933
 -0.49258448  0.44286765 -0.44964295  0.49121325  0.1485508 ]
proj_coord is: [ 0.2141331  -0.26311459 -0.271546    0.28621838  0.5996947   0.37630481
 -0.49485332  0.41180197 -0.63344482  0.42804156  0.25482255]
original coord is: [ 0.06829804 -0.70647407 -0.36583206 -0.63169017  0.82542057  0.00568988
 -0.8585683   0.02925623 -1.8528106   0.33435946 -0.55834055]
proj_coord is: [ 0.0541222   0.15565878 -0.06692923 -0.03312431  0.47517253  0.20233612
  0.00496483  0.1965512  -0.1644913  -0.00637388  0.31188765]
proj_c

/home/wanxinli/.local/lib/python3.9/site-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


outliet_indices is: []
subspace dimension is: 3


In [9]:
""" 
Run
(1) NB normalization + cMDS 
(2) QE normalization + cMDS 

To derive the axes data
"""  

axes_files = ["hmp_target_NB_MDS_cMDS_axes.txt", "hmp_target_QE_MDS_cMDS_axes.txt"] # put NB before QE
data_files = ["hmp_v13lqphylotypeQuantNB_target_rs_c.csv", "hmp_v13lqphylotypeQuantE_target_rs_c.csv"]

for i in range(len(axes_files)):
    axes_file = axes_files[i]
    data_file = data_files[i]

    print("======== QE/NB normalization + MDS + cMDS ========")
    data_path = os.path.join(data_dir, data_file)
    axes_output_path = os.path.join(output_dir, axes_file)

    df_hmp = np.loadtxt(data_path, delimiter=",")
    hmp_dis_sq=squareform(pdist(df_hmp))

    # Plot cMDS embedding using the pairs of axis from the four most significant axes 
    enforce_dim = subspace_dims[i] # enforcing the dimension to be consistent with nSimplices QE+nsimplices+cMDS
    embedding = MDS(n_components=enforce_dim) 
    corr_coord = embedding.fit_transform(hmp_dis_sq)
    corr_dis_sq=squareform(pdist(corr_coord))
    _, _, Xe = cMDS(corr_dis_sq)

    np.savetxt(axes_output_path, Xe, fmt='%f')

======== QE/NB normalization + MDS + cMDS ========


/home/wanxinli/.local/lib/python3.9/site-packages/sklearn/manifold/_mds.py:601: UserWarning: The MDS API has changed. ``fit`` now constructs an dissimilarity matrix from data. To use a custom dissimilarity matrix, set ``dissimilarity='precomputed'``.
  warnings.warn(
/home/wanxinli/.local/lib/python3.9/site-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


======== QE/NB normalization + MDS + cMDS ========


/home/wanxinli/.local/lib/python3.9/site-packages/sklearn/manifold/_mds.py:601: UserWarning: The MDS API has changed. ``fit`` now constructs an dissimilarity matrix from data. To use a custom dissimilarity matrix, set ``dissimilarity='precomputed'``.
  warnings.warn(
/home/wanxinli/.local/lib/python3.9/site-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


In [12]:
""" 
Plot pairwise result
"""

figure_files = ["hmp_target_NB_nSimplices_cMDS.png", "hmp_target_QE_nSimplices_cMDS.png", "hmp_target_NB_MDS_cMDS.png", "hmp_target_QE_MDS_cMDS.png"]
axes_files = ["hmp_target_NB_nSimplices_cMDS_axes.txt", "hmp_target_QE_nSimplices_cMDS_axes.txt", "hmp_target_NB_MDS_cMDS_axes.txt", "hmp_target_QE_MDS_cMDS_axes.txt"]
titles = ["NB+nSimplices", "QuantE+nSimplices", "NB+MDS", "QuantE+MDS"]
num_axes = 3 # show pairwise 2D plot to decompose the 3D plot

for i in range(len(figure_files)):
    figure_file = figure_files[i]
    axes_file = axes_files[i]
    title = titles[i]

    print("======== plot pairwise 2D plot (subset) ========")
    Xe = np.loadtxt(os.path.join(output_dir, axes_file))
    print(Xe.shape)
    for first_dim in range(num_axes):
        for second_dim in range(first_dim+1, num_axes):
            plt.figure()
            # only plot stool (blue), ears (black), throat (pink) points 

            stool_indices = [i for i, e in enumerate(colors) if e == 'cornflowerblue']
            ears_indices = [i for i, e in enumerate(colors) if e == 'orange']

            plt.scatter(Xe[stool_indices, second_dim], \
                Xe[stool_indices, first_dim], s=5, c='cornflowerblue', label = "STOOL")
            plt.scatter(Xe[ears_indices, second_dim], \
                Xe[ears_indices, first_dim], s=5, c='orange', label = "VAGINA")


            # for i in range(Xe.shape[0]):
            #     plt.scatter(Xe[i, second_dim], Xe[i, first_dim], s=5, c=colors[i])
            plt.legend()
            plt.title(title, size=10)   
            plt.savefig(os.path.join(output_dir, figure_file[:-4]+"_"+str(first_dim)+"_"+str(second_dim)+".png"))
            

======== plot pairwise 2D plot (subset) ========
(330, 162)
======== plot pairwise 2D plot (subset) ========
(330, 161)
======== plot pairwise 2D plot (subset) ========
(330, 159)
======== plot pairwise 2D plot (subset) ========
(330, 158)
